In [ ]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
from kamino import image_utils as iu

%matplotlib ipympl

In [ ]:
# https://maker.pro/raspberry-pi/tutorial/grid-detection-with-opencv-on-raspberry-pi
plt.close("all")

impath = Path("..") / "data" / "key_card_4.jpg"
image = cv2.imread(str(impath))

plt.figure()
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

max_area = 0
c = 0
for contour in contours:
    area = cv2.contourArea(contour)
    if area < 25000:
        if area > max_area:
            max_area = area
            best_cnt = contour
            # image = cv2.drawContours(image, contours, c, (0, 255, 0), 3)
    c += 1

mask = np.zeros((gray.shape), np.uint8)
cv2.drawContours(mask, [best_cnt], 0, 255, -1)
cv2.drawContours(mask, [best_cnt], 0, 0, 2)
plt.figure()
plt.imshow(mask)
plt.title("Mask")

out = np.zeros_like(gray)
out[mask == 255] = gray[mask == 255]
plt.figure()
plt.imshow(cv2.cvtColor(out, cv2.COLOR_BGR2RGB))
plt.title("New Image")

blur = cv2.GaussianBlur(out, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)
contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

c = 0
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 100 and area < 1000:
        x, y, w, h = cv2.boundingRect(contour)
        roi = image[y : y + h, x : x + w]
        cv2.rectangle(image, (x, y), (x + w, y + h), (36, 255, 12), 2)
    c += 1

plt.figure()
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Final Image")

plt.show()